## Named entity recognition with spacy

In [1]:
import spacy
import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer
from collections import Counter

# nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('en_core_web_md')

In [2]:
# Import the email modules we'll need
import glob
import email
import mailparser
from email import policy
from email.parser import BytesParser

path = '../datawe/raw/Email_Classification/*'
email_types = glob.glob(path)
appendFilesData = []
file_raw_data = [] 
for folder in email_types:
    files = glob.glob(folder+"/*.txt")
    email_type = folder.split('\\')[1]
    for name in files:
        try:
            with open(name) as fp:
                raw_data = fp.read()
                file_raw_data.append(raw_data)
                msg = mailparser.parse_from_string(raw_data)
                appendFilesData.append({
                    "to":msg.to,
                    "from":msg.from_,
                    "subject":msg.subject,
                    "date":msg.date,
#                     "sent":msg["Sent"],
#                     "importance":msg["Importance"],
                    "content":  msg.body,
                    "class_to_exec":email_type,
                })
         
        except IOError as exc:
            print('Exception')

            
#creating pandas dataframe
data = pd.DataFrame(appendFilesData)

In [3]:
# data

In [4]:
body = data["content"][:1]
entityData = []
for cnt in body:
    entityCh = []
    entity = nlp(cnt)
    for token in entity.ents:
        entityData.append({"text":token.text, "label":token.label_})
#     entityData.append(entityCh)


In [5]:
entityDF = pd.DataFrame(entityData)

In [6]:
entityDF["label"].value_counts()

CARDINAL    2
PERSON      2
DATE        2
FAC         1
Name: label, dtype: int64

In [7]:
type(entityDF["text"][1])

str

In [8]:
# text = nlp('. '.join(np.array(data["content"])))

In [9]:
# for token in text:
#     print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#           token.shape_, token.is_alpha, token.is_stop)

In [10]:
# entityDtl = []
# for token in text.ents:
#     if len(token.text.rstrip()) > 0:
#         entityDtl.append({"label":token.label_, "text":token.text})

In [11]:
# entyDF = pd.DataFrame(entityDtl)
# entyDF

In [12]:
# type(entyDF["text"][3])

In [13]:
# import spacy
# nlp = spacy.load("en_core_web_sm")

In [14]:
sentence = """
Dear Sir/ Madam,

 

Greetings of the day!

 

As per your request, I am pleased to assist you with the reservation
details as follows:

 

*         Reservation #: 75677SB034081

*         Guest Name: Saroj Kumar

*         Hotel: Taj Dubai

*         Dates of Stay: Tue Nov 20 - Sun Nov 25, 2018

*         Room: Luxury Burj View Room King Bed

*         Rate: Stay a Bit Longer – AED 6500.61

*         Inclusions: Breakfast, complimentary shuttle to Dubai mall at
fixed timings and Wi-Fi upto 4 devices 

*         Cancellation: Reservations must be cancelled by 2PM - 1 day
prior to arrival to avoid a penalty of 1 night charge plus taxes

 

Please do not hesitate to contact us if we may be of further assistance
to you.

 

Warm Regards,

Diana Chettiar

Taj Reservations Worldwide Team





 """

In [15]:


# # en_core_web_sm = spacy.load('en_core_web_sm')
# # en_core_web_md = spacy.load('en_core_web_md')
# # en_core_web_lg = spacy.load('en_core_web_lg')
# # en_vectors_web_lg = spacy.load('en_vectors_web_lg')



# # en_core_web_sm_doc = en_core_web_sm(sentence)
# # en_core_web_md_doc = en_core_web_md(sentence)
# # en_core_web_lg_doc = en_core_web_lg(sentence)
# # en_vectors_web_lg_doc = en_vectors_web_lg(sentence)

# doc = nlp(sentence)

# entityDtl = []
# for token in text.ents:
#     if len(token.text.rstrip()) > 0:
#         entityDtl.append({"label":token.label_, "text":token.text})

In [16]:
# pd.DataFrame(entityDtl)

entities = {"GPE":"Location",
 "ORG":"organization",
 "PERSON":"Name",
 "DATE":"Date",
 "TIME" : "Time",
 "CARDINAL": "CARDINAL"
}
stemmer = SnowballStemmer("english").stem

In [17]:
text = nlp(sentence)
entityDtl = []
for token in text.ents:
    wrd = token.text.strip()
    label_ =  token.label_
#     if len(wrd) > 0:
#         entityDtl.append({"label":token.label_, "text":token.text.strip()})

    if len(wrd) > 0 and label_ in entities and ((label_ == "PERSON" and wrd.replace(' ','').isalpha() and stemmer(wrd) == wrd.lower()) or label_ != "PERSON"):
        entityDtl.append({"label":token.label_, "text":token.text.strip()})

entityDtlDF = pd.DataFrame(entityDtl)

In [19]:
# names = entityDtlDF[entityDtlDF["label"] == "PERSON"]["text"]

# # names.

# stemmer = SnowballStemmer("english").stem
# stem_free = [stemmer(stem) for stem in names if stem]
# stem_free
items = [x.text for x in text.ents]
Counter(items).most_common(3)

[('\n\n', 5), ('Sir/ Madam', 1), ('\n\n \n\n', 1)]

In [20]:
entityDtlDF

,label,text
0,CARDINAL,75677SB034081
1,PERSON,Guest
2,PERSON,Saroj Kumar
3,PERSON,Taj Dubai
4,DATE,"Tue Nov 20 - Sun Nov 25,"
5,DATE,2018
6,CARDINAL,6500.61
7,GPE,Dubai
8,DATE,2PM - 1 day
9,TIME,1 night


In [221]:
((label_ == "PERSON" and wrd.isalpha() and stemmer(wrd) == wrd) or label_ != "PERSON")

False

In [225]:
stemmer(wrd) == wrd.lower()

True

In [226]:
wrd.isalpha()

False

In [224]:
stemmer(wrd)

'diana chettiar'

In [130]:
# import spacy

# nlp = spacy.load('en_core_web_md')  # make sure to use larger model!
tokens = nlp(u'dog cat banana')

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

dog dog 1.0
dog cat 0.53906965
dog banana 0.28761008
cat dog 0.53906965
cat cat 1.0
cat banana 0.4875216
banana dog 0.28761008
banana cat 0.4875216
banana banana 1.0


In [215]:
help("")

Help on class str in module builtins:

class str(object)
 |  str(object='') -> str
 |  str(bytes_or_buffer[, encoding[, errors]]) -> str
 |  
 |  Create a new string object from the given object. If encoding or
 |  errors is specified, then the object must expose a data buffer
 |  that will be decoded using the given encoding and error handler.
 |  Otherwise, returns the result of object.__str__() (if defined)
 |  or repr(object).
 |  encoding defaults to sys.getdefaultencoding().
 |  errors defaults to 'strict'.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __format__(...)
 |      S.__format__(format_spec) -> str
 |      
 |      Return a formatted version of S as described by format_spec.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getatt